In [ ]:
# gist from: https://gist.github.com/jkuruzovich/b8485a368f80a3b88df46326cf54bbce

# Facebook Graph API Example in Python
# by James Thornton, http://jamesthornton.com

# Facebook API Docs
# https://developers.facebook.com/docs/graph-api/using-graph-api#reading

# Get Your Facebook Access Token Here...
# https://developers.facebook.com/tools/explorer/145634995501895/?method=GET&path=me

# Before running this script...
# Set your Facebook Access Token as an environment variable in your terminal:
# $ export ACCESS_TOKEN={YOUR ACCESS TOKEN}

# To download this script, use the curl command...
# $ curl -O https://gist.githubusercontent.com/espeed/11114604/raw/a233d14604ea44f9d29af02cd2768b91caaad7af/fbme.py

# To run this script, use the python command to execute the script in your terminal...
# $ python fbme.py

import os
import json
import requests
import urllib
import pprint
import subprocess
import ast
from time import sleep

import configparser
Config = configparser.ConfigParser()
Config.read('config.cnf')

import facebook

To generate app access token head over to https://developers.facebook.com and create a new app. Enter the APP_ID and APP_SECRET in your `config.cnf`

In [ ]:
# Programmatic User Access Token for App
FACEBOOK_APP_ID = Config.get('facebook', 'app_id')
FACEBOOK_APP_SECRET = Config.get('facebook', 'app_secret')

# Retrieve App Access Token
def get_fb_token(app_id, app_secret):           
    payload = {'grant_type': 'client_credentials', 'client_id': app_id, 'client_secret': app_secret}
    try:
        r = requests.post('https://graph.facebook.com/oauth/access_token?', params = payload)
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        print(e)
        return
    
    r = ast.literal_eval(r.text)['access_token']
    
    print("Generated access token: " + r)
    return r

In [ ]:
ACCESS_TOKEN = get_fb_token(FACEBOOK_APP_ID, FACEBOOK_APP_SECRET)

In [ ]:
graph = facebook.GraphAPI(ACCESS_TOKEN)

In [ ]:
# DOI -> FB engagement
def get_facebook(doi):
    query_url = 'http://dx.doi.org/%s' % doi
    url = 'v2.10/?' + urllib.parse.urlencode({'id': query_url, 'fields': 'engagement'})
    try: 
        response = graph.request(url)
        return response['engagement']
    except: 
        return False 
    
# DOI -> altmetric.com
def get_altmetric(doi):
    global counter
    url = 'http://api.altmetric.com/v1/fetch/doi/%s' % doi
    params = {'key': Config.get('altmetric', 'key')}

    try: 
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()['counts']['facebook']
        else:
            return False
    except: 
        return False 

In [ ]:
print(get_facebook('10.1590/S1413-92511998000200004'))
print(get_altmetric('10.1590/S1413-92511998000200004'))

In [ ]:
query_url = 'http://www.tandfonline.com/eprint/QK4zmjpieTnYdMD72ntV/full'
url = 'v2.10/?' + urllib.parse.urlencode({'id': query_url, 'fields': 'engagement'})
graph.request(url)

In [ ]:
# a URL that does not resolve nicely because of internal link resolver
doi = '10.1016/j.plefa.2017.04.005'
r = requests.head('http://dx.doi.org/%s' % doi, allow_redirects=True)
print(r.url)

print(get_facebook(doi))